In [2]:
!git clone https://github.com/Team-M1/badwords-classifier-train -b DW
%cd badwords-classifier-train
!pip install -r requirements.txt

Cloning into 'badwords-classifier-train'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 67 (delta 29), reused 50 (delta 12), pack-reused 0
Unpacking objects: 100% (67/67), done.
/content/badwords-classifier-train
     |████████████████████████████████| 264 kB 5.5 MB/s 
     |████████████████████████████████| 272 kB 35.4 MB/s 
     |████████████████████████████████| 2.6 MB 32.5 MB/s 
     |████████████████████████████████| 918 kB 42.2 MB/s 
     |████████████████████████████████| 118 kB 44.7 MB/s 
     |████████████████████████████████| 243 kB 811 kB/s 
     |████████████████████████████████| 50 kB 6.1 MB/s 
     |████████████████████████████████| 895 kB 40.2 MB/s 
     |████████████████████████████████| 3.3 MB 37.1 MB/s 
     |████████████████████████████████| 636 kB 47.0 MB/s 
     |████████████████████████████████| 829 kB 37.2 MB/s 
     |████████████████████████████████| 1.3 MB 42

In [3]:
from google.colab import drive
drive.mount('/content/drive')

path_to_drive = "/content/drive/MyDrive/Colab Notebooks"

Mounted at /content/drive


# 시드 통일하기

In [15]:
import torch
from transformers import set_seed


set_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# 모델 불러오기

In [10]:
model_config = {
    "num_labels": 3,
    "id2label": {0: 0, 1: 1, 2: 2},
    "label2id": {0: 0, 1: 1, 2: 2}
}

In [12]:
from transformers import ElectraForSequenceClassification
from tokenization_kocharelectra import KoCharElectraTokenizer


model = ElectraForSequenceClassification.from_pretrained("monologg/kocharelectra-small-discriminator", **model_config)
tokenizer = KoCharElectraTokenizer.from_pretrained("monologg/kocharelectra-small-discriminator", model_max_length=512)

Some weights of the model checkpoint at monologg/kocharelectra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/kocharelectra-small-discriminator and are newly initializ

# 하이퍼 파라미터 설정

In [ ]:
num_classes = 3

# 원하는 대로 고쳐서 사용
batch_size = 64
lr = 5e-5  # 0.00005
epochs = 15

'0.000050'

In [ ]:
# 옵티마이저와 스케줄러
# 원하는 대로 고쳐서 사용

# from torch.optim import AdamW
# from transformers import AdamW, get_linear_schedule_with_warmup


# LENGTH_OF_TRAIN_DATA = 40242
# num_training_steps = ((LENGTH_OF_TRAIN_DATA - 1) // batch_size + 1) * epochs
# optimizer = AdamW(model.parameters(), lr=lr)
# scheduler = get_linear_schedule_with_warmup(optimizer, int(num_training_steps * 0.1), num_training_steps)

# 만약 이 스케줄러를 사용할 경우,
# optimizer.step() 바로 다음에
# scheduler.step()을 호출해야 함

# f1 score 계산하기

In [ ]:
# 1. torchmetrics 사용
# requirements에 넣어놨으니 따로 설치할 필요 없음

# from torchmetrics import F1


# f1_score = F1(num_classes=num_classes)

# # 검증 단계에서 사용
# for inputs, labels in val_loader:
#     # 대충 코드
#     output = model(inputs)  # 대충 아웃풋
#     pred = torch.argmax(output, dim=1)
#     batch_f1 = f1_score(pred, labels)
#     print(batch_f1)

# f1 = f1_score.compute()
# print(f1)

In [ ]:
# 2. datasets 사용
# requirements에 넣어놨으니 따로 설치할 필요 없음

# from datasets import load_metric


# f1_score = load_metric("f1")

# for inputs, labels in val_loader:
#     # 대충 코드
#     output = model(inputs)
#     pred = torch.argmax(output, dim=1)
#     f1_score.add_batch(predictions=pred, references=labels)

# f1 = f1_score.compute()
# print(f1)

# 훈련

In [14]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score


def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred, average="micro")

    return {"accuracy": accuracy, "f1": f1}

In [16]:
from data_loader import get_data_loaders


train_data, val_data, test_data = get_data_loaders(tokenizer, return_loader=False)

Using custom data configuration default-879a8a3f1b1ce670


0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-879a8a3f1b1ce670/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.


  0%|          | 0/40242 [00:00<?, ?ex/s]

  0%|          | 0/4472 [00:00<?, ?ex/s]

  0%|          | 0/5000 [00:00<?, ?ex/s]

In [21]:
from transformers import TrainingArguments, EarlyStoppingCallback

from trainer import ImbalancedSamplerTrainer


args = TrainingArguments(
    output_dir=path_to_drive,
    overwrite_output_dir=True,
    evaluation_strategy="steps",
    num_train_epochs=15,
    logging_steps=1000, 
    save_strategy="steps",
    save_steps=1000,
    seed=42,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
)

trainer = ImbalancedSamplerTrainer(
    model=model,
    args=args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [22]:
trainer.train()

***** Running training *****
  Num examples = 40242
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 5031


Epoch,Training Loss,Validation Loss


The following columns in the evaluation set  don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: content.
***** Running Evaluation *****
  Num examples = 4472
  Batch size = 8


NameError: ignored